## Advanced DataFrames Practice

In [1]:
# imports
import numpy as np
import pandas as pd
from env import host, user, password, get_db_url

np.random.seed(123)

In [2]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))


In [3]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})


In [4]:
url = get_db_url('employees')

In [5]:
# query employees table from employees database
emp_query = '''

SELECT * 
FROM employees e;'''


In [6]:
employees = pd.read_sql(emp_query, url)
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [7]:
tit_query = '''

SELECT * 
FROM titles'''

In [8]:
titles = pd.read_sql(tit_query, url)

In [9]:
titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [10]:
employees.shape

(300024, 6)

In [11]:
titles.shape

(443308, 4)

In [12]:
# there are a lot more titles than employees. This makes sense if many
# employees have changed titles during their tenure

In [13]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [14]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


In [15]:
# how many unique titles are there? 7
titles.title.nunique()

7

In [16]:
# the oldest date in the to_date column is 3/1/1985
# it appears that the protocol is to use 1/1/9999 as a proxy for 'presently' or
# some future time
titles.to_date.min(), titles.to_date.max()

(datetime.date(1985, 3, 1), datetime.date(9999, 1, 1))

#### Indexing and Subsetting
- Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator [].
- I will demonstrate concrete examples of indexing using the indexing operator [] alone and with the .loc and .iloc attributes below.


In [17]:
# Choose only two columns for my subset.

df[['name', 'classroom']]



,name,classroom
0,Sally,A
1,Jane,B
2,Suzie,A
3,Billy,B
4,Ada,A
5,John,B
6,Thomas,A
7,Marie,A
8,Albert,A
9,Richard,A


In [18]:
# can pass a boolean Series to the indexing operator as a selector
bools = df.name.str.startswith('A')
bools

0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9     False
10    False
11     True
Name: name, dtype: bool

In [19]:
df[bools]

,name,math,english,reading,classroom
4,Ada,77,92,98,A
8,Albert,92,62,87,A
11,Alan,92,62,72,A


- We can use the .loc attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! The .loc attribute's indexing is inclusive and uses an index label, not position.

 - this looks like `df.loc[row_indexer, column_indexer]` in general form

In [20]:
# select all the rows and a subset of the columns. Note .loc is inclusive.
df.loc[:, 'math':'reading']

,math,english,reading
0,62,85,80
1,88,79,67
2,94,74,95
3,98,96,88
4,77,92,98
5,79,76,93
6,82,64,81
7,93,63,90
8,92,62,87
9,69,80,94


In [21]:
# I can use a boolean Series as a selector with .loc, too, but I can choose rows and columns.

df.loc[bools, 'name': 'reading']


,name,math,english,reading
4,Ada,77,92,98
8,Albert,92,62,87
11,Alan,92,62,72


- We can use the `.iloc` attribute to select specific rows and colums by index position. .iloc does not accept a boolean Series as a selector like `.loc` does. It takes in integers representing index position and is NOT inclusive.
-  basic syntax: `df.iloc[row_indexer, column_indexer]`




In [22]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]


,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A


In [23]:
# rows 0, 1, 2 and columns 1 and 2 (excluding 0, 3, and 4)
df.iloc[:3, 1:3]


,math,english
0,62,85
1,88,79
2,94,74


#### Aggregating
- The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.



In [24]:
# can pass lists of columns to the indexer and a list of aggregation functions to .agg
df[['english', 'reading', 'math']].agg(['mean', 'min', 'max'])


,english,reading,math
mean,77.666667,86.5,84.833333
min,62.000000,67.0,62.000000
max,99.000000,98.0,98.000000


#### .groupby

The `.groupby()` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [25]:
df.groupby('classroom').math.max()

classroom
A    94
B    98
Name: math, dtype: int64

- We can group by multiple columns as well. To demonstrate, we'll create a boolean column named passing_math, then group by the combination of our new feature, passing_math, and the classroom and calculate the average reading grade and the number of individuals in each subgroup.



`np.where()` we can create a column based on a condition using np.where()
- general syntax: `np.where(condition, this_where_True, this_where_False)`


In [26]:
df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')

In [27]:
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing


In [28]:
grade_groups = df.drop(columns='name').groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

mean  count
passing_math classroom                  
failing      A          87.000000      2
passing      A          87.166667      6
             B          85.250000      4

In [29]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups


avg_reading_grade  count_of_students
passing_math classroom                                      
failing      A                  87.000000                  2
passing      A                  87.166667                  6
             B                  85.250000                  4

`.transform()`
- The .transform method can be used to produce a series with the same length of the original dataframe where each value represents the aggregation from the subgroup resulting from the .groupby.
- This is great when we want to create a new column for the original df with aggregated group data for each individual record. 




In [30]:
df.assign(avg_math_score_by_classroom=df.groupby('classroom').math.transform('mean'))

,name,math,english,reading,classroom,passing_math,avg_math_score_by_classroom
0,Sally,62,85,80,A,failing,82.625
1,Jane,88,79,67,B,passing,89.250
2,Suzie,94,74,95,A,passing,82.625
3,Billy,98,96,88,B,passing,89.250
4,Ada,77,92,98,A,passing,82.625
5,John,79,76,93,B,passing,89.250
6,Thomas,82,64,81,A,passing,82.625
7,Marie,93,63,90,A,passing,82.625
8,Albert,92,62,87,A,passing,82.625
9,Richard,69,80,94,A,failing,82.625


`.describe()`
- we can chain a .describe() onto a groupby to get summary statistics for the grouped data

In [31]:
df.groupby('classroom').reading.describe()

,count,mean,std,min,25%,50%,75%,max
classroom,,,,,,,,
A,8.0,87.125,8.887190,72.0,80.75,88.5,94.25,98.0
B,4.0,85.250,12.392874,67.0,82.75,90.5,93.00,93.0


#### Merging and Joining
- Pandas provides several ways to combine dataframes together. We will look at two of them below:



`pd.concat()`
- This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.



- Default is set to row-wise concatenation using an outer join.

`pd.concat(objs, axis=0, join='outer')`


- When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names.



In [32]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1


,a
0,1
1,2
2,3


In [33]:
df2

,a
0,4
1,5
2,6


In [34]:
pd.concat([df1, df2])

,a
0,1
1,2
2,3
0,4
1,5
2,6


Note that the indices are preserved on the resulting dataframe; we could set the ignore_index parameter to True if we wanted these to be sequential.



In [35]:
concat_df1 = pd.concat([df1, df2], ignore_index=True)
concat_df1

,a
0,1
1,2
2,3
3,4
4,5
5,6


In [36]:
concat_df2 = pd.DataFrame({'b': [1, 2, 3, 4, 5, 6]})
concat_df2


,b
0,1
1,2
2,3
3,4
4,5
5,6


In [37]:
pd.concat([concat_df1, concat_df2], axis=1)


,a,b
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6


`.merge()`
- This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.

- In addition, the how keyword argument is used to define what type of JOIN we want to do; as we saw above, inner is the default setting.

`# df.merge default settings for commonly used parameters.`

`left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)`

How does changing the default argument of the how parameter change my resulting DataFrame?

how == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.



In [38]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [39]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


`.merge()` will allow us to specify left_on and right_on to indicate the columns that are the keys used to merge the dataframes together.

- In addition, the how keyword argument is used to define what type of JOIN we want to do; as we saw above, inner is the default setting.
- For demonstration purposes, setting the indicator parameter to True, which will create a column indicating whether the merge key appears in the left_only, right_only or both DataFrames.

In [40]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


- Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of _x to any columns in the left dataframe that are duplicated, and _y to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:



In [41]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)


,id,employee,role_id,role
0,1.0,bob,1.0,admin
1,2.0,joe,2.0,author
2,3.0,sally,3.0,reviewer
3,4.0,adam,3.0,reviewer
4,5.0,jane,NaN,NaN
5,6.0,mike,NaN,NaN
6,NaN,NaN,4.0,commenter


## Exercises II

#### 1. Copy the users and roles DataFrames from the examples above.



#### 2. What is the result of using a right join on the DataFrames?



In [42]:
users.merge(roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


Here we see that the right join returned every record that had a valid role_id, then also returned the role that didn't have any match and didn't return jane and mike who didn't have roles assigned.

#### 3. What is the result of using an outer join on the DataFrames?

In [43]:
users.merge(roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


- in the outer join, every record is represented whether it has a match in the other df or not. 

#### 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?



In [44]:
user_no_role = users.drop(columns='role_id')
roles_no_id = roles.drop(columns='id')
user_no_role.merge(roles_no_id)

,id,name


there are no results returned, probably because there are no matches.

In [45]:
user_no_role.merge(roles_no_id, how='outer')

,id,name
0,1.0,bob
1,2.0,joe
2,3.0,sally
3,4.0,adam
4,5.0,jane
5,6.0,mike
6,NaN,admin
7,NaN,author
8,NaN,reviewer
9,NaN,commenter


with an outer join and no keys, the records are still returned one frame appended to the other

#### 5. Load the mpg dataset from PyDataset.



In [46]:
from pydataset import data
mpg = data('mpg')

#### 6. Output and read the documentation for the mpg dataset.



In [47]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [48]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [49]:
mpg.sort_values('fl')

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
107,honda,civic,1.8,2008,4,auto(l5),f,24,36,c,subcompact
32,chevrolet,k1500 tahoe 4wd,6.5,1999,8,auto(l4),4,14,17,d,suv
123,jeep,grand cherokee 4wd,3.0,2008,6,auto(l5),4,17,22,d,suv
213,volkswagen,jetta,1.9,1999,4,manual(m5),f,33,44,d,compact
223,volkswagen,new beetle,1.9,1999,4,auto(l4),f,29,41,d,subcompact
...,...,...,...,...,...,...,...,...,...,...,...
110,hyundai,sonata,2.4,1999,4,manual(m5),f,18,27,r,midsize
111,hyundai,sonata,2.4,2008,4,auto(l4),f,21,30,r,midsize
112,hyundai,sonata,2.4,2008,4,manual(m5),f,21,31,r,midsize
37,chevrolet,malibu,3.6,2008,6,auto(s6),f,17,26,r,midsize


#### 7. How many rows and columns are in the dataset?



In [50]:
print(f'There are {mpg.shape[1]} columns and {mpg.shape[0]} rows.')

There are 11 columns and 234 rows.


#### 8. Check out your column names and perform any cleanup you may want on them.



In [51]:
old_cols = mpg.columns.tolist()
new_cols = ['manufacturer', 'model', 'displacement', 'year', 'number_cylinders', 
           'transmission_type', 'drive_type', 'city_mileage', 'highway_mileage',
           'fuel_type', 'class']
col_dict = dict(zip(old_cols, new_cols))

In [52]:
mpg.rename(columns=col_dict, inplace=True)

#### 9. Display the summary statistics for the dataset.



In [53]:
mpg.describe()

,displacement,year,number_cylinders,city_mileage,highway_mileage
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [54]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer       234 non-null    object 
 1   model              234 non-null    object 
 2   displacement       234 non-null    float64
 3   year               234 non-null    int64  
 4   number_cylinders   234 non-null    int64  
 5   transmission_type  234 non-null    object 
 6   drive_type         234 non-null    object 
 7   city_mileage       234 non-null    int64  
 8   highway_mileage    234 non-null    int64  
 9   fuel_type          234 non-null    object 
 10  class              234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


#### 10. How many different manufacturers are there?



In [55]:
mpg.manufacturer.nunique()

15

#### 11. How many different models are there?



In [56]:
mpg.model.nunique()

38

#### 12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.



In [57]:
mpg['mileage_difference'] = mpg.highway_mileage - mpg.city_mileage

In [58]:
mpg.head()

,manufacturer,model,displacement,year,number_cylinders,transmission_type,drive_type,city_mileage,highway_mileage,fuel_type,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10


#### 13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.



In [59]:
mpg['average_mileage'] = round(((mpg.highway_mileage + mpg.city_mileage) / 2), 2)

In [60]:
mpg.head()

,manufacturer,model,displacement,year,number_cylinders,transmission_type,drive_type,city_mileage,highway_mileage,fuel_type,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0


#### 14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.



In [61]:
mpg['is_automatic'] = mpg.transmission_type.str.contains('auto')

In [62]:
mpg.head()

,manufacturer,model,displacement,year,number_cylinders,transmission_type,drive_type,city_mileage,highway_mileage,fuel_type,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True


#### 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?



In [63]:
mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False)

manufacturer
honda         28.500000
volkswagen    25.074074
hyundai       22.750000
subaru        22.428571
audi          22.027778
toyota        21.720588
pontiac       21.700000
nissan        21.346154
chevrolet     18.447368
ford          16.680000
mercury       15.625000
jeep          15.562500
dodge         15.540541
lincoln       14.166667
land rover    14.000000
Name: average_mileage, dtype: float64

#### 16. Do automatic or manual cars have better miles per gallon?



In [64]:
mpg.groupby('is_automatic').average_mileage.mean()

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

## III. Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.



`pd.crosstab()` is a function that acts sort of like a pivot table creating a dataframe with unique values from one column serving as row names and unique values from another column serving as column names. The values in the df are counts of where the records(rows) from the original df have each combination of possibilities.

In [65]:
# going back to the student grades dataframe
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing


In [66]:
pd.crosstab(df.passing_math, df.classroom)

classroom,A,B
passing_math,,
failing,2,0
passing,6,4


We can also view subtotals with the margins set to True.

In [67]:
pd.crosstab(df.passing_math, df.classroom, margins=True)

classroom,A,B,All
passing_math,,,
failing,2,0,2
passing,6,4,10
All,8,4,12


Or we can view the numbers as percentages of the total by setting normalize to True.

In [68]:
pd.crosstab(df.passing_math, df.classroom, normalize=True).round(3)

classroom,A,B
passing_math,,
failing,0.167,0.000
passing,0.500,0.333


#### `.pivot_table()`
this method can also be used to to create our summary. This method produces output similar to an excel pivot table. We must supply 4 things here:

- which values will make up the rows (the index)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (aggfunc); if we omit this, mean will be used by default.

For an example using the pivot_table method, we'll calculate the average math grade for the combination of classroom and passing_math status.

In [69]:
df.pivot_table(index='classroom', columns='passing_math', values='math')

passing_math,failing,passing
classroom,,
A,65.5,88.333333
B,NaN,89.250000


So here we can see the average grades for each classroom divided by whether the students are failing or passing.

#### `.map()` 
This method in pandas is used to map values of a Series from one set to another. It's a convenient way to transform or replace values in a Series (a single column in a DataFrame) based on a given mapping. Here's a summary of its functionality:

- Mapping Values: You can pass a dictionary, function, or Series as an argument to .map(). The method then applies the mapping to each element in the Series.
- Dictionary Mapping: If a dictionary is passed, .map() replaces the values in the Series with the corresponding values from the dictionary. It's useful for categorical data conversion or remapping.
- Function Mapping: When a function is passed, it's applied to each value in the Series. This is useful for more complex transformations.
- Series Mapping: If a Series is passed, it's similar to a dictionary mapping but allows for more flexibility, as the Series index is matched with the Series being mapped.
- Handling Missing Values: If a value from the original Series doesn't exist in the mapping, the method will return a NaN (Not a Number) for that value.
- In-Place Transformation: .map() does not modify the original Series; instead, it returns a new Series with the mapped values. You need to assign the result back to the DataFrame if you want to store the changes.

In [70]:
# Here we'll create a dataframe that represents various orders at a restaurant.

n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

# let's look at a random sample of the orders df:
orders.sample(10)


,drink,meal
0,Tea,Pad Thai
38,Water,Curry
11,Tea,Curry
4,Tea,Curry
16,Water,Curry
2,Tea,Yakisoba Noodle
22,Water,Yakisoba Noodle
5,Water,Pad Thai
19,Tea,Pad Thai
15,Water,Yakisoba Noodle


The .map method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named bill. Let's do this step-by-step.

In [71]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}


In [72]:
# now we use .map() to perform the calculation of the prices for each meal by adding the food and drink costs.
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

orders.sample(10)


,drink,meal,bill
6,Water,Pad Thai,10
31,Water,Curry,11
7,Water,Yakisoba Noodle,9
32,Water,Yakisoba Noodle,9
0,Tea,Pad Thai,12
22,Water,Yakisoba Noodle,9
38,Water,Curry,11
15,Water,Yakisoba Noodle,9
21,Tea,Yakisoba Noodle,11
29,Tea,Yakisoba Noodle,11


Let's take a look at how many orders have each combination of meal and drink:

In [73]:
pd.crosstab(orders.drink, orders.meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,3,4,5
Water,9,10,9


In [74]:
pd.crosstab(orders.drink, orders.meal, normalize=True, margins=True)

meal,Curry,Pad Thai,Yakisoba Noodle,All
drink,,,,
Tea,0.075,0.10,0.125,0.3
Water,0.225,0.25,0.225,0.7
All,0.300,0.35,0.350,1.0


In [75]:
# let's find out the average bill amount for each combination
orders.pivot_table(index='drink', columns='meal', values='bill')


meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


It's interesting to note that we could find the same information with a multi-level group by:



In [76]:
orders.groupby(['drink', 'meal']).bill.mean()

drink  meal           
Tea    Curry              13.0
       Pad Thai           12.0
       Yakisoba Noodle    11.0
Water  Curry              11.0
       Pad Thai           10.0
       Yakisoba Noodle     9.0
Name: bill, dtype: float64

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand.



#### Transposing
The rows and columns of a DataFrame can be transposed by using the .T method.

In [77]:
df.T


,0,1,2,3,4,5,6,7,8,9,10,11
name,Sally,Jane,Suzie,Billy,Ada,John,Thomas,Marie,Albert,Richard,Isaac,Alan
math,62,88,94,98,77,79,82,93,92,69,92,92
english,85,79,74,96,92,76,64,63,62,80,99,62
reading,80,67,95,88,98,93,81,90,87,94,93,72
classroom,A,B,A,B,A,B,A,A,A,A,B,A
passing_math,failing,passing,passing,passing,passing,passing,passing,passing,passing,failing,passing,passing


In [78]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
math,12.0,84.833333,11.134168,62.0,78.50,90.0,92.25,98.0
english,12.0,77.666667,13.371158,62.0,63.75,77.5,86.75,99.0
reading,12.0,86.500000,9.643651,67.0,80.75,89.0,93.25,98.0


## Exercises III

1. Use your get_db_url function to help you explore the data from the chipotle database.

In [82]:
url = get_db_url('chipotle')

In [83]:
query = '''

SELECT * FROM orders;
'''

In [85]:
ch = pd.read_sql(query, url)

In [86]:
ch.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


In [89]:
ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


In [91]:
# item_price is an object so let's cast that to a float
ch.item_price = ch.item_price.str.strip('$').astype('float')

In [93]:
# id is redundant with the index so dropping that
ch.drop(columns='id', inplace=True)

2. What is the total price for each order?


In [94]:
# Let's take this to mean "Create a column 'order_total' that shows the total price for each order."
ch.assign(order_total=ch.groupby('order_id').item_price.transform('sum'))

,order_id,quantity,item_name,choice_description,item_price,order_total
0,1,1,Chips and Fresh Tomato Salsa,nan,2.39,11.56
1,1,1,Izze,[Clementine],3.39,11.56
2,1,1,Nantucket Nectar,[Apple],3.39,11.56
3,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,11.56
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,16.98
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,23.50
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,23.50
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25,28.75
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75,28.75


3. What are the most popular 3 items?

In [97]:
ch.groupby('item_name').quantity.sum().sort_values(ascending=False).head(3)

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

4. Which item has produced the most revenue?

In [99]:
ch.groupby('item_name').item_price.sum().sort_values(ascending=False).head(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

5. Join the employees and titles DataFrames together.



In [101]:
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [102]:
titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [104]:
# each employee has held one or more job titles so we want to right join on titles and should end up
# with a df that is the same length as the titles df
employees.shape[0], titles.shape[0]

(300024, 443308)

In [108]:
employees.merge(titles, how='right', on='emp_no')

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


In [106]:
# output length is correct, also can see various titles for the same emp no.

6. For each title, find the hire date of the employee that was hired most recently with that title.



In [109]:
emp_tit = employees.merge(titles, how='right', on='emp_no')

In [114]:
emp_tit.groupby('title').hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)



In [124]:
# lets get the dept_emp table as a df
url_emp = get_db_url('employees')
dept_query = '''
SELECT * FROM dept_emp
LEFT JOIN departments
USING(dept_no);
'''
depts = pd.read_sql(dept_query, url_emp)

In [125]:
depts.head()

,dept_no,emp_no,from_date,to_date,dept_name
0,d005,10001,1986-06-26,9999-01-01,Development
1,d007,10002,1996-08-03,9999-01-01,Sales
2,d004,10003,1995-12-03,9999-01-01,Production
3,d004,10004,1986-12-01,9999-01-01,Production
4,d003,10005,1989-09-12,9999-01-01,Human Resources


In [126]:
emp_tit.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [127]:
# if we inner join on emp_no and from_date, we should have the set of records where there are 
# departments and titles appropriately matched by employee and time.
emp_tit.merge(depts, on=['emp_no', 'from_date'], how='inner')

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date_x,dept_no,to_date_y,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,9999-01-01,Development
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,d007,9999-01-01,Sales
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,d004,9999-01-01,Production
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,d004,9999-01-01,Production
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,Staff,1989-09-12,1996-09-12,d003,9999-01-01,Human Resources
...,...,...,...,...,...,...,...,...,...,...,...,...
300051,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12,Engineer,1997-06-02,9999-01-01,d004,9999-01-01,Production
300052,499996,1953-03-07,Zito,Baaz,M,1990-09-27,Engineer,1996-05-13,2002-05-13,d004,9999-01-01,Production
300053,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29,d005,9999-01-01,Development
300054,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27,d002,9999-01-01,Finance


In [129]:
# ok, that looks good. I'll assign it
emp_tit_dept = emp_tit.merge(depts, on=['emp_no', 'from_date'], how='inner')

In [130]:
# now I just need to do the crosstab
pd.crosstab(emp_tit_dept.title, emp_tit_dept.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,1,7669,0,0,0,5959,1501,0,0
Engineer,0,52819,0,0,0,42283,10615,5,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,2,14957,0,0,0,11993,3103,3,0
Senior Staff,3388,1,3239,3267,3308,0,0,3296,10095
Staff,13351,1,13245,13614,13311,1,0,13494,40374
Technique Leader,0,7603,0,0,0,6071,1478,0,0
